In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATA_DIR='/home/HDD6TB/datasets/emotions/EmotiW/engagement/'

In [3]:
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import mord

from collections import defaultdict
import os
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
compare_filenames=lambda x: int(os.path.splitext(x)[0])

## PyTorch

In [5]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

print(f"Torch: {torch.__version__}")
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 1.10.2+cu113
True


In [6]:
if False:
    model_name='enet_b2_8'
    IMG_SIZE=260 #224 #
else:
    model_name='enet_b0_8_best_afew'
    IMG_SIZE=224
PATH='../../models/affectnet_emotions/'+model_name+'.pt'
test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
np_transforms = transforms.Compose(
    [
        transforms.ToPILImage(None),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

feature_extractor_model = torch.load(PATH)

AttributeError: 'NoneType' object has no attribute 'message_types_by_name'

In [ ]:
feature_extractor_model.classifier=torch.nn.Identity()
feature_extractor_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0

In [ ]:
def get_features(data_dir):
    filename2features={}
    for filename in tqdm(os.listdir(data_dir)):
        frames_dir=os.path.join(data_dir,filename)
        X_global_features=[]
        imgs=[]
        faces_only=[fn for fn in os.listdir(frames_dir) if 'noface' not in fn]
        for img_name in sorted(faces_only, key=compare_filenames):
            img = Image.open(os.path.join(frames_dir,img_name))
            img_tensor = test_transforms(img)
            if img.size:
                imgs.append(img_tensor)
                if len(imgs)>=64:        
                    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)

                    imgs=[]
            

        if len(imgs)>0:        
            features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
            features=features.data.cpu().numpy()
            #print(features.shape)

            if len(X_global_features)==0:
                X_global_features=features
            else:
                X_global_features=np.concatenate((X_global_features,features),axis=0)

            imgs=[]

        #print(X_global_features.shape,X_feats.shape,X_scores.shape)
        filename2features[filename]=X_global_features
    return filename2features

print(test_transforms)
filename2features_val=get_features(os.path.join(DATA_DIR,'frames/faces/mtcnn_aligned/validation/')) 

Compose(
    Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


  0%|          | 0/48 [00:00<?, ?it/s]

## Predict 

In [ ]:
import csv

def create_dataset(filename2features):
    x = []
    y = []
    ind=0
    for fn in filename2features:
        cur_features=filename2features[fn]
        total_features=None
        
        total_features=np.std(cur_features, axis=0)
        x.append(total_features)
        y.append(video2label[fn])
    x=np.array(x)
    y=np.array(y)
    print(x.shape,y.shape)
    return x,y

video2label={}
with open(os.path.join(DATA_DIR,'Engagement_Labels_Engagement.csv'), mode='r') as csvfile:
    labels_reader = csv.reader(csvfile, delimiter='\t')
    for i,row in enumerate(labels_reader):
        if i==0:
            #print('first:',row)
            continue
        videoname,label=row[0],float(row[1])
        video2label[videoname]=label
video2label['subject_87_Vid_3']=video2label['subject_77_Vid_6']

x_test, y_test = create_dataset(filename2features_val)

(48, 1280) (48,)


In [ ]:
#EngageWild
reg_coef,reg_intercept = np.load('mord_reg_enet_b0_8_best_afew.npy',allow_pickle=True)

In [ ]:
y_pred=np.dot(x_test,reg_coef)+reg_intercept
y_pred = np.round(y_pred)
y_pred = np.clip(y_pred, 0, 3)
print(y_pred)
int2float={0:0,1:0.33,2:0.66,3:1} #EngageWild: disengaged, barely engaged, engaged, highly engaged.
y_pred_float=np.array([int2float[y] for y in y_pred])
print("MSE:",((y_test-y_pred_float)**2).mean())

[2. 1. 2. 3. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 3. 2. 1. 2. 2. 2. 3. 2. 1. 2.
 2. 2. 2. 1. 2. 1. 2. 2. 2. 2. 3. 2. 2. 1. 2. 3. 2. 2. 3. 2. 2. 2. 2. 1.]
MSE: 0.05626458333333333


In [ ]:
#DAISEE
with open('daisee_svm_norm_enet_b0_8_best_afew.pkl', 'rb') as handle:
    cls=pickle.load(handle)

In [ ]:
x_test_norm=preprocessing.normalize(x_test,norm='l2')
y_pred = cls.predict(x_test_norm)
print(y_pred)
int2float={0:0,1:0.33,2:0.66,3:1} # very low, low, high, very high
y_pred_float=np.array([int2float[y] for y in y_pred])
print("MSE:",((y_test-y_pred_float)**2).mean())

[2 2 2 3 2 2 2 2 2 3 2 2 2 2 2 2 2 3 2 2 3 2 2 3 2 3 2 3 2 3 3 3 2 2 3 3 2
 2 2 2 2 3 2 2 2 2 2 2]
MSE: 0.11622083333333333
